In [59]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [60]:
dataset = pd.read_csv('/content/sample_data/formspring_data.csv',delimiter='\t',quoting=3)

In [61]:
import re
bag_of_words = []
def clean_words(words):
    cleaned_words = re.sub('[^a-zA-Z]',' ',words.lower())
    cleaned_words = cleaned_words.split()
    words = [word for word in cleaned_words if not word in set(stopwords.words("english"))]
    words = ' '.join(words)
    bag_of_words.append(words)

In [62]:
df = dataset[['bully1','ans1']]

In [63]:
df = df.dropna()

In [64]:
holder = df['bully1'] != 'None'
df = df[holder]

In [65]:
df

,bully1,ans1
28,fake a$$ friend!!</3,No
38,Ur a fuckin idiot!,Yes
56,Hello bitch!,Yes
62,"""""""""""""""why you such a bitch?""""""""""""""",Yes
68,Who the hell are yuh!!! And who tha fuck is mi...,Yes
...,...,...
12842,rather cheat or be cheated on?,Yes
12865,we both lifted up our legs and she went flying...,Yes
12871,piece of shit,Yes
12883,"""""""Referring to the other user as a """"""""Dumbas...",Yes


In [66]:
df.ans1 = df.ans1.replace(("No","Yes"),(0,1))

In [68]:
df['ans1'] = df['ans1'].astype(int)

In [74]:
for word in df['bully1']:
    clean_words(word)

In [88]:
X,y = np.array(bag_of_words),df['ans1'].values

In [91]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [92]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(X_train)
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [94]:
X_train

<977x1410 sparse matrix of type '<class 'numpy.int64'>'
	with 3197 stored elements in Compressed Sparse Row format>

In [96]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [97]:
input_dim = X_train.shape[1]
model = Sequential()
model.add(Dense(25,activation='relu',input_dim=input_dim))
model.add(Dense(1,activation='sigmoid'))

In [98]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [99]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 25)                35275     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 26        
Total params: 35,301
Trainable params: 35,301
Non-trainable params: 0
_________________________________________________________________


In [109]:
history = model.fit(X_train,y_train,batch_size=25,epochs=70,verbose=2,validation_data=(X_test,y_test))

Epoch 1/70
40/40 - 0s - loss: 0.0373 - accuracy: 0.9898 - val_loss: 0.7665 - val_accuracy: 0.8160
Epoch 2/70
40/40 - 0s - loss: 0.0373 - accuracy: 0.9898 - val_loss: 0.7657 - val_accuracy: 0.8160
Epoch 3/70
40/40 - 0s - loss: 0.0372 - accuracy: 0.9898 - val_loss: 0.7671 - val_accuracy: 0.8160
Epoch 4/70
40/40 - 0s - loss: 0.0373 - accuracy: 0.9898 - val_loss: 0.7735 - val_accuracy: 0.8160
Epoch 5/70
40/40 - 0s - loss: 0.0372 - accuracy: 0.9898 - val_loss: 0.7705 - val_accuracy: 0.8160
Epoch 6/70
40/40 - 0s - loss: 0.0373 - accuracy: 0.9898 - val_loss: 0.7685 - val_accuracy: 0.8160
Epoch 7/70
40/40 - 0s - loss: 0.0372 - accuracy: 0.9898 - val_loss: 0.7727 - val_accuracy: 0.8160
Epoch 8/70
40/40 - 0s - loss: 0.0372 - accuracy: 0.9898 - val_loss: 0.7738 - val_accuracy: 0.8160
Epoch 9/70
40/40 - 0s - loss: 0.0372 - accuracy: 0.9898 - val_loss: 0.7735 - val_accuracy: 0.8160
Epoch 10/70
40/40 - 0s - loss: 0.0372 - accuracy: 0.9898 - val_loss: 0.7749 - val_accuracy: 0.8160
Epoch 11/70
40/40 -

In [106]:
def pre(words):
    words = cv.transform(words).toarray()
    prediction = model.predict(words)
    return prediction

In [116]:
# Scores
y_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score,recall_score,f1_score
# Using treshold of .5
y_pred = (y_pred > 0.5)


In [117]:
acc = accuracy_score(y_test,y_pred)
recall = recall_score(y_test,y_pred)
f1 = f1_score(y_test,y_pred)

print("Accuracy = ",acc)
print("Recall = ",recall)
print("F1 Score = ",f1)

Accuracy =  0.8159509202453987
Recall =  0.7883817427385892
F1 Score =  0.8636363636363636
